# Image Classification with CNN
The idea of this model is to class the animals into their own species based in a few features  

### Load Dataset and Import First Libraries

In [ ]:
#import nvidia.cudnn
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from sklearn.metrics import confusion_matrix
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.optimizers import Adam

"""
Download dataset
"""
# Load CIFAR-10 dataset and split into training data and tests data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

### Normalize Data / Clean up (Filter Animal classes)

In [ ]:
# Animal classes in CIFAR-10: 2 (bird), 3 (cat), 4 (deer), 5 (dog), 6 (frog), 7 (horse)
animal_classes = [2, 3, 4, 5, 6, 7]
input_shape = (32, 32, 3)

# Creating a boolean mask to filter only animal classes
train_mask = np.isin(y_train, animal_classes)
test_mask  = np.isin(y_test, animal_classes)

x_train_animals = x_train[train_mask.flatten()]
y_train_animals = y_train[train_mask.flatten()]
x_test_animals  = x_test[test_mask.flatten()]
y_test_animals  = y_test[test_mask.flatten()]

# Relabel classes
label_map = {2: 0, 3: 1, 4: 2, 5: 3, 6: 4, 7: 5}
y_train_animals = np.vectorize(label_map.get)(y_train_animals)
y_test_animals  = np.vectorize(label_map.get)(y_test_animals)

# Normalize Data
x_train_animals = x_train_animals.astype("float32") / 255.0
x_test_animals  = x_test_animals.astype("float32") / 255.0

## Visualisize images
# read call lables
class_names = np.unique(y_train_animals)
fig, axes = plt.subplots(5, 10, figsize=(15, 15))

for i in range(5):
    # Get indices for images of class i
    indices = np.where(y_train_animals == i)[0]
    # Randomly select 10 indices from the class
    random_indices = np.random.choice(indices, size=10, replace=False)
    
    for j, idx in enumerate(random_indices):
        # Plot the image in the appropriate subplot
        axes[i, j].imshow(x_train_animals[idx])
        axes[i, j].axis('off')  # Hide axis ticks
        if j == 0:
            axes[i, j].set_ylabel(class_names[i], fontsize=12)  # Label the class

plt.subplots_adjust(wspace=0.1, hspace=0.3)
plt.show()

y_train_animals = to_categorical(y_train_animals, 6)
y_test_animals  = to_categorical(y_test_animals, 6)

# Verify if shapes correspond to expectations
print("x_train_animals shape:", x_train_animals.shape)
print("y_train_animals shape:", y_train_animals.shape)
print("x_test_animals shape:", x_test_animals.shape)
print("y_test_animals shape:", y_test_animals.shape)

### Build Model And Train It

In [ ]:
from keras.backend import clear_session
clear_session()

In [ ]:
# init model
vgg_model = Sequential()

# add Convolutional-Layer 
vgg_model.add(Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=input_shape))
vgg_model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))

# add MaxPooling-Layer
vgg_model.add(MaxPooling2D((2, 2), padding='same'))

# add 2 more Convolutional-Layer 
vgg_model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
vgg_model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))

# improve training speed and stability
vgg_model.add(BatchNormalization())


# add MaxPooling-Layer
vgg_model.add(MaxPooling2D((2, 2), padding='same'))

# Flatten output
vgg_model.add(Flatten())

# add Dense-Layer with 128 units
vgg_model.add(Dense(128, activation='relu'))

# add dropout after dense-layer to  prevent overfitting
vgg_model.add(Dropout(0.5))

# add classification layer (5 classes)
vgg_model.add(Dense(6, activation='softmax'))

# model summary
vgg_model.summary()

In [ ]:
# train settings
batch_size = 512
num_classes = 6
epochs = 45


adam_opt = Adam(learning_rate=0.001)


# compile the model
vgg_model.compile(loss='categorical_crossentropy', optimizer=adam_opt, metrics=['accuracy'])

# train model
history_m3 = vgg_model.fit(x_train_animals, y_train_animals, 
                    batch_size=batch_size,
                    epochs=epochs,
                    validation_data=(x_test_animals, y_test_animals))


# predict output for test data
predictions = vgg_model.predict(x_test_animals)

print(predictions.shape)
predictions = np.argmax(predictions, axis=1)

# print confusion matrix
gt = np.argmax(y_test_animals, axis=1)
confusion_matrix(gt, predictions)